In [ ]:
Add latex macros$$\newcommand{\V}[1]{{\boldsymbol{#1}}}\newcommand{mean}[1]{{\mathbb{E}\left[#1\right]}}\newcommand{var}[1]{{\mathbb{V}\left[#1\right]}}\newcommand{covar}[2]{\mathbb{C}\text{ov}\left[#1,#2\right]}\newcommand{corr}[2]{\mathbb{C}\text{or}\left[#1,#2\right]}\newcommand{argmin}{\mathrm{argmin}}\def\rv{z}\def\reals{\mathbb{R}}\def\pdf{\rho}\def\rvdom{\Gamma}\def\coloneqq{\colon=}\newcommand{norm}{\lVert #1 \rVert}\def\argmax{\operatorname{argmax}}\def\ai{\alpha}\def\bi{\beta}\newcommand{\dx}[1]{\;\mathrm{d}#1}$$


# Sparse Grid Interpolation

We will use the Genz benchmark to illustrate how to use a sparse grid
as a surrogate for uncertainty quantification and sensitivity analysis

First we must load the benchmark


In [ ]:
import numpy as np
from pyapprox.benchmarks import setup_benchmark
from pyapprox import surrogates
from pyapprox.analysis import visualize
from pyapprox import util
import matplotlib.pyplot as plt
benchmark = setup_benchmark('genz', nvars=2, test_name="oscillatory")
np.random.seed(1)

#We can set a maximum number of samples using the options dictionary
options = {"max_nsamples": 100, "verbose": 0}
approx = surrogates.adaptive_approximate(
    benchmark.fun, benchmark.variable, "sparse_grid", options).approx

Plot the sparse grid samples with



In [ ]:
sparse_grid_samples = approx.get_samples()
util.plot_2d_samples(sparse_grid_samples, marker='o')
plt.show()

We can estimate the error in the surrogate using some validation samples.



In [ ]:
validation_samples = benchmark.variable.rvs(100)
validation_values = benchmark.fun(validation_samples)
approx_values = approx(validation_samples)
error = np.linalg.norm(validation_values-approx_values, axis=0)/np.sqrt(
    validation_values.shape[0])
print(f"The RMSE error is {error}")

and building a kernel density estimator. Lets first just plot the marginal
PDFs of the output



In [ ]:
surrogate_samples = benchmark.variable.rvs(10000)
approx_values = approx(surrogate_samples)
visualize.plot_qoi_marginals(approx_values)
plt.show()